# Assignment group 4: Machine learning and regression

## Module B _(62 pts)_ Exploring Classifier Transferability
### Data sets

__Data Set 1:__ There's a lot more than e-mail text out there, and malicious SPAM-like text-based deception is pervasive in other domains. One domain of particular interest to a few companies is called _opinion SPAM_, in which product and business reviews are spoofed, either to help or hurt a business.

An interesting data set for purposes of studying opinion SPAM was produced by a researcher named Myle Ott. In addition to collecting real reviews on hotels from the web and TripAdvisor, Ott et al. ran Amazon Mechanical Turk surveys to have real people write both positive and negative fake reviews of the hotels:

- http://myleott.com/op-spam.html

The goal with the data set was to train computers to detect which reviews were real vs. fake. These are provided in the following nested file structure:

- `./data/op_spam_v1.4/negative_polarity/deceptive_from_MTurk/fold[1-5]/*.txt`
- `./data/op_spam_v1.4/positive_polarity/deceptive_from_MTurk/fold[1-5]/*.txt`
- `./data/op_spam_v1.4/negative_polarity/truthful_from_Web/fold[1-5]/*.txt`
- `./data/op_spam_v1.4/positive_polarity/truthful_from_TripAdvisor/fold[1-5]/*.txt`

__Data Set 2:__ The big picture of what we're trying to do here is train an Opinion SPAM classifier on the _curated_ __Data Set 1__, and apply it to get an idea of how prolific SPAM is on this completely different, _real-world_ hotel [booking website's](booking.com) data. The data from this website live in the assignment's data directory, too:

- `./data/Hotel_Reviews.csv`
    
and were taken from [Kaggle](https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe).

__B1.__ _(2 pts_) To load the Op SPAM data we'll be using `sklearn`, but as a requirement we'll need a full list of all the different review files in the data set. To compile a list of file paths, review the datas directory structure and use the `glob` module's `.glob(regex)` method to output a list of all `all_files` matching the provided `regex` pattern.

When this is complete, print the first 5 files to show your code's function.

In [1]:
import glob

## a regex-like patter that specifies all possible file locations
file_paths = "./data/op_spam_v1.4/*/*/*/*.txt"

## grab all file paths matching the pattern
all_files = glob.glob(file_paths)

## let's see what the first two file path
## that we got are
for filename in all_files[0:2]:
    print(filename)

./data/op_spam_v1.4/positive_polarity/deceptive_from_MTurk/fold2/d_talbott_9.txt
./data/op_spam_v1.4/positive_polarity/deceptive_from_MTurk/fold2/d_talbott_8.txt


__B2.__ _(3 pts)_ Since this is supervised learning, we'll neeed labels, too. To construct, use a regex match on `all_files`. In particular, since we're doing sentiment classification, utilize the word 'positive_polarity' in the file path to indicate a positve label (of value `1`) and otherwise use a negative label (value `0`). Store these values in a `np.array()` called `labels.

When this is done, compute and print the size of positive and negative portions of the data set and discuss the imbalance you observe in the response box below. 

_Response._ 

In [2]:
import re
import numpy as np
## make an empty list for our class labels
labels = []
## loop through all files
for filename in all_files:
    ## if the file path has the word "positive_polarity"
    ## the it's spam (\ell = 1)
    if re.search("positive_polarity", filename):
        labels.append(1)
    else:
        labels.append(0)
        
labels = np.array(labels)

__B3.__ _(3 pts)_ Now, `import` `sklearn`'s TDM-maker `CountVectorizer` from `sklearn.feature_extraction.text`. Initialize an instance of 
- `CountVectorizer(input = 'filename')` 

and called `vectorizer`, apply its `.fit()` and `.transform()` methods to `all_files` to produce a `TDM`.

When this is complete, exhibit its shape, and be sure to apply `TDM.toarra()` to convert the matrix to a dense representation.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

## initialize the vectorizer
vectorizer = CountVectorizer(input = 'filename')

## tokenize and build a vocab that spans all files
## note, this establishes the TDM's tracked words and their indices
vectorizer.fit(all_files)

# create the TDM (it's sparse)
TDM = vectorizer.transform(all_files)

# let's check out a little to see if it worked!
print(TDM.shape)
print(type(TDM))
print(TDM.toarray())

(1600, 9571)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


__B4.__ _(2 pts)_ Now, use `train_test_split` to split the `TDM` and `labels` into $75\%$ training and $25\%$ test sets, importing the function from `sklearn.model_selection`. Also, be sure to use use `random_state = 0`.

In [4]:
from sklearn.model_selection import train_test_split

## split the data into 3/4 (training) and 1/4 (test/evaluation)
TDM_train, TDM_test, labels_train, labels_test = train_test_split(
    TDM, labels, 
    test_size=0.25, 
    random_state=0
)

## let's just check to see how many elements there are
print(len(labels_train), len(labels_test))

1200 400


__B5.__ _(5pts)_ Now, `import`, initialize, and `.fit()` a binary classifer of your choosing (from __Chapter 8.__) with `sklearn` on the training data split. After training, apply and print `.predict()` and `.score()` to review the model's accuracy.

In [5]:
from sklearn.naive_bayes import MultinomialNB

## initialize naive bayes
classifier = MultinomialNB(alpha=0.1)

## run the classifier with separate training and test sets
classifier.fit(TDM_train, labels_train)

print(classifier.score(TDM_test, labels_test))

0.9375


__B6.__ _(5 pts)_ Now, determine precision, recall, and $F_1$ for the classifier's performance on the test set. Do these results provide any different information as compared to accuracy? If not, why do you think? Provide discussion in the markdown cell below.

_Response._

Accuracy is not always an ideal measure of a model's success. For example, if you're trying to classify a population that represents 1% of the total population, if you build a classifier that always yields a negative response, you'll end up with 99% accuracy, but a terrible model. The precision, recall, and $F_1$ scores give more insight into the false negatives and false positives of the model. It is worth noting that since the data is split into roughly evenly-sized categories here that accuracy is actually fairly useful, but it doesn't hurt to check with extra measures.

In [6]:
from sklearn.metrics import f1_score, precision_score, recall_score

predictions_test = classifier.predict(TDM_test)

## look at our accuracy
print("Here's the accuracy: ")
print(classifier.score(TDM_test, labels_test))
print("")

## compare accuract to the model's precision, recall, and F1
print("Here's precision, recall, and F1")
print(precision_score(predictions_test, labels_test))
print(recall_score(predictions_test, labels_test))
print(f1_score(predictions_test, labels_test))

Here's the accuracy: 
0.9375

Here's precision, recall, and F1
0.9512195121951219
0.9285714285714286
0.9397590361445782


__B7.__ _(2 pts)_ Let's see how well our sentiment polarity classifier does on a different data set:

- `./data/Hotel_Reviews.csv`

which was hosted on a Kaggle competition, but came from from Booking.com:

- https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

There's a decent description of the data there, where it seems a customer can comment with positive and negative reviews, in parallel. To get started, load these data in with pandas, print out the column names and identify (in the markdown cell, below) which have the positive and the negative reviews.

_Response._ We want `'Positive_Review'` and `'Negative_Review'`.

In [7]:
import pandas as pd

## load the hotel data
hotel_data = pd.read_csv('./data/Hotel_Reviews.csv')

## print the column names
print(hotel_data.columns)

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')


__B8.__ _(1 pts)_ Sometimes, a reviewer won't leave a positive or negative review in one of the categories. However, what's left is not a conventional N/A or anything. Refer back to the data dictionary:

- https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

and determine what we should match for to filter out any missing/null reviews.

_Response._ It looks like there is a `'No Negative'` or `'No Positive'` if they're not there.

__B9.__ _(7 pts)_ Use your observation from __B8.__ to create a single list with all of the non null reviews, as well as a parallel list of labels: $1$s (for positive review texts) and $0$s (for the negative review texts).

In [8]:
## make empty lists for reviews and labels
booking_reviews = []
booking_labels = []

## filter out all positive reviews as a list
positive_reviews = list(
    hotel_data["Positive_Review"][
        hotel_data["Positive_Review"] != 'No Positive'
    ]
)
## extend the all reviews list with the positive reviews
booking_reviews.extend(positive_reviews)
## extend the labels list with 1s for each positive review
booking_labels.extend([1 for review in positive_reviews])

## filter out all negative reivews as a list
negative_reviews = list(
    hotel_data["Negative_Review"][
        hotel_data["Negative_Review"] != 'No Negative'
    ]
)
## extend the all reviews list with the negative reviews
booking_reviews.extend(negative_reviews)
## extend the labels list with 0s for each negative review
booking_labels.extend([0 for review in negative_reviews])

__B10.__ _(2 pts)_ How many positive and negatives were there? Does this data set have a class imbalance? Specifically, determine the percentage of reviews that were positive and comment on the presence of any imbalance in the markdown cell below.

_Response._ It looks like there's a few more positives than negatives, and not any _serious_ class imbalance.

In [9]:
## print the fraction of positive reviews
print(sum(booking_labels)/len(booking_labels))

0.5529851090313955


__B11.__ _(5 pts)_ Use `CountVectorizer()` again&mdash;now to create a TDM for the new hotel data. Note: You must use the same initialized vectorizer from __B3.__, i.e., after is has run `.fit()`. So, here you must start from the `'.transform()'` step. If you re-initialize the vectorizer, you will wind up with a different vocabulary! Note: you also have to change the input format with `vectorizer.input`. It was equal to `'filename'` which would create a TDM by a list of files. Now we want it to work off of a list of strings. This will work if we set:
- `'vectorizer.input = content'`

In [10]:
# change the input format of the CountVectorizer()
# instance to accept a list of string-documents as input
vectorizer.input = 'content'

# create the TDM (it's sparse)
booking_TDM = vectorizer.transform(booking_reviews)

# let's check out a little to see if it worked!
print(booking_TDM.shape)
print(type(booking_TDM))
print(booking_TDM.toarray())

(867640, 9571)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


__B12.__ _(5 pts)_ Apply your Classifier to this new, Booking.com TDM and compute accuracy, precision, recall, and $F_1$. What do you notice? Is there any more of a class imbalance now? Comment in the markdown cell below.

_Response._

Since there is an imbalance of positive reviews, if the classifier is skewed towards a positive prediction, these performance numbers could be an inaccurate overestimate of model performance.

In [11]:
## figure out our test-set predictions
booking_predictions = [
    1 if x > 0.5 else 0 
    for x in classifier.predict_proba(booking_TDM)[:, 1]
]

## check out percent correct on the test set
booking_numcorrect = len([
    1 for pred, label in zip(booking_predictions, booking_labels) 
    if pred == label
])

## look at our accuracy
print("Here's the accuracy: ")
print(100*booking_numcorrect/len(booking_labels))
print("")

## compare accuract to the model's precision, recall, and F1
print("Here's precision, recall, and F1")
print(precision_score(booking_predictions, booking_labels))
print(recall_score(booking_predictions, booking_labels))
print(f1_score(booking_predictions, booking_labels))

Here's the accuracy: 
84.74401825641971

Here's precision, recall, and F1
0.9172287157768366
0.8260779790251401
0.8692703883854719


__B13.__ _(2 pts)_  Compare these results with the results from __B6__. Is the performance better or worse in some areas (e.g., precision vs. recall) than others? Do you think our sentiment polarity classifier transferred well from the one Opintion SPAM dataset to this one from Booking.com? Place your discussion in the markdown box below.

_Response._ 

Precision is still pretty high, but recall is lower. So, $F_1$ and accuracy are lower, but it's not because the positive predictions are less reliable, rather because we're finding fewer of the positive reviews. This is probably because of the domain transfer. 

__B14.__ _(3 pts)_ Go back to the Opinion SPAM data and rebuild the _SPAM_ (no longer sentiment polarity) labels for that dataset's classification, in particular using the patter `deceptive` inside of the file names to produce positive-valued (`1`) labels, and `0`s, otherwise.

In [12]:
## make an empty list for our class labels
deception_labels = []
## loop through all files
for filename in all_files:
    ## if the file path has the word "positive_polarity"
    ## the it's spam (\ell = 1)
    if re.search("deceptive", filename):
        deception_labels.append(1)
    else:
        deception_labels.append(0)

__B15.__ _(2 pts)_ Now, train your classifier on _all_ of the Opinion SPAM labels. Note: you _must_ initialize a new classifier in order to classify _SPAM_, instead of polarity. However, we can just reuse our `TDM` from __B3__.

In [13]:
## initialize naive bayes
SPAM_classifier = MultinomialNB(alpha=0.1)

## train the classifier with on the Opinon SPAM data
SPAM_classifier.fit(TDM, deception_labels)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

__B16.__ _(3 pts)_ Run the classifier you just trained on the new hotel reviews data set. Make classification at a threshold of $0.5$ and report the percentage of the new data set that our classifier thinks is SPAM. 

In [14]:
## figure out our test-set predictions
booking_predictions = [
    1 if x > 0.5 else 0 
    for x in SPAM_classifier.predict_proba(booking_TDM)[:, 1]
]

## print the portion of booking.com reviews found to be SPAM
print(sum(booking_predictions)/len(booking_predictions))

0.2264257065142225


__B17.__ (2 pts) Interpret the output percentage from __B16__. Is this a big number? If correct, what would it mean for Booking.com? Do you our classification was a reliable assessment? Why or why not? Place your discussion in the markdown cell, below.

_Response._ 

It seems like there is some chance our classifier is working, since we were able to have some decent performance transfer with the polarity classification. This (22.64%) is a _big_ portion of the reviews from Booking.com, but very different from the 50% proportion in the original Opinion SPAM dataset.

__C18.__ _(2 pts)_ Sort the Booking.com reviews by their prediction probabilities from high to low. Either use `sorted()` on a list of `(probability, review)` tuples, or create a pandas data frame with the two columns and use the `.sort_values()` method.

In [15]:
ReviewProb_tupleSort = sorted(zip(
    list(SPAM_classifier.predict_proba(booking_TDM)[:, 1]), booking_reviews
), reverse = True)

__B19.__ _(2 pts)_ We really don't have SPAM labels for the Booking.com data. So, inspect the first few most and least spammy reviews. What observations can you draw? Do you see any qualitative differences between the most and least spammy reviews? Do you think the classifier is working?  Place you discussion in the markdown cell, below.

_Response._ It looks like the most spammy reviews might be _extremely_ postive or negative, but not mixed, while the least spammy reviews appear to have very detailed information. In other words, once you started a spammy review there wasn't much that was surprising. This seems pretty reasonable output from the classifier.

In [16]:
print(ReviewProb_tupleSort[5])

(1.0, ' Hotel Room was disgusting there was pubic hair and toenail clippings all over the floor in the bathroom urine stains underneath the toilet seat dust on all surfaces and a massive build up in all corners of the room it was like it had not been cleaned for months I complained to the reception staff and they rudely just moved me to another room without any sort of apology When I had entered my new room I saw that it was in state as the room before however it was even dirtier I was tried and I couldn t be bothered to wait and complain anymore so I just had to put up with the outcome I went to the local shop and purchased some cleaning products and on my return I started to detail clean the room I had cleaned the main bedroom and was happy with my outcome iI felt clean again it was until I detailed cleaned the bathroom that made me want to be sick with the dirt that I came across I have complained to the hotel group and not happy with my overall compensation they offered me a free n

__B20.__ (2 pts) What aspects of our classifier could we modify to potentially improve our SPAM classifier's performance? Specifically, discuss the potential effects to this experiment in selecting or transforming our features, or optimizing any criteria for our predictions in the markdown cell, below.

_Respnose._

There are several features that we could add to the classification apart from just term frequency. It seems that the spam reviews are on average shorter than the ham reviews. Thus, calculating the length of each review could be useful in the creation of another significant feature. 

In addition, since we've created a term document matrix, it would be easy to also use TF-IDF to create another feature for the model. Also, as always, it may prove beneficial to normalize any numerical features that we end up with. 

Also, we just randomly selected the hyperparamter value of $\alpha = 0.1$. It could be informative and potentially useful to perform a hyperparameter scan to find the best value of $\alpha$.

Another aspect we can modify is to make classification at a different threshold, this would make fewer reviews been classfied as spam.  

__B21.__ _(2 pts)_ How could we get an evaluation out of this experiment and _really_ know if the classifer is working? What would we have to do with the Booking.com data in order to get a strong sense of our performance on SPAM? Is there _any_ reasonable labeling of this data that we could come up with, or would we have to get some new data that we have more control over? Place your discussion in the markdown cell, below.

_Response._

We need to lablel the booking reviews such that we will realy know if the classifer is working. The existing data could be labeled for spam without acquiring newer data. However,  we would have to perform fact checking to know if we are actually . providing valuable labels.

We could also use the available column "Total_Number_of_Reviews_Reviewer_Has_Given" to get a sense of how to label the review as deceptive or not. If this solution doesn't seem to make sense, we then need to get some new data such as crossing check the reviewer and review date with the hotel's guest history. 